<a href="https://colab.research.google.com/github/zjzsu2000/CMPE297_Sec49AdvanceDL/blob/master/Assignment_6/Assignment_6__TFX_interactivecontext_based_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE297-49 Assignment 6: TFX interactivecontext based colab


## Installing and Importing

### Upgrade Pip

To avoid upgrading Pip in a system when running locally, check to make sure that we're running in Colab.  Local systems can of course be upgraded separately.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

     |████████████████████████████████| 1.5MB 24.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


### 
**Note: In Google Colab,you must restart the runtime at first time upgrade**

In [ ]:
!pip install -q -U --use-feature=2020-resolver tfx

     |████████████████████████████████| 2.1 MB 12.2 MB/s 
     |████████████████████████████████| 8.9 MB 43.6 MB/s 
     |████████████████████████████████| 151 kB 71.6 MB/s 
     |████████████████████████████████| 146 kB 69.7 MB/s 
     |████████████████████████████████| 2.0 MB 63.3 MB/s 
     |████████████████████████████████| 829 kB 79.7 MB/s 
     |████████████████████████████████| 173 kB 68.0 MB/s 
     |████████████████████████████████| 114 kB 71.7 MB/s 
     |████████████████████████████████| 267 kB 82.4 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 91 kB 13.1 MB/s 
     |████████████████████████████████| 91 kB 12.5 MB/s 
     |████████████████████████████████| 91 kB 13.0 MB/s 
     |████████████████████████████████| 91 kB 13.6 MB/s 
     |████████████████████████████████| 91 kB 12.7 MB/s 
     |████████████████████████████████| 91 kB 13.2 MB/s 
     |████████████████████████████████| 169 kB 78.7 MB/s 
     |████████████████

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen, Evaluator, ExampleValidator, Pusher
from tfx.components import ResolverNode, SchemaGen, StatisticsGen, Trainer, Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.utils.dsl_utils import external_input


%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.3.0
TFX version: 0.25.0


### Set up pipeline paths

In [ ]:
_tfx_root = tfx.__path__[0]

#mnist data root
_mnist_root =  os.path.join(_tfx_root, 'examples/mnist')

#cifar10
_cifar10 =  os.path.join(_tfx_root, 'examples/cifar10')

#imdb
_imdb = os.path.join(_tfx_root, 'examples/imdb')

#iris
_iris = os.path.join(_tfx_root, 'examples/iris')

#mnist_pipeline
_mnist_root = os.path.join(_tfx_root, 'examples/mnist_pipeline')

#probablity_of_customer_compliants
_probablity =  os.path.join(_tfx_root, 'examples/probablity_of_customer_compliants')

_serving_model_dir_mnist = os.path.join(tempfile.mkdtemp(), 'serving_model/mnist_simple')
_serving_model_dir_cifar10 = os.path.join(tempfile.mkdtemp(), 'serving_model/cifar10_simple')
_serving_model_dir_imdb = os.path.join(tempfile.mkdtemp(), 'serving_model/imdb_simple')
_serving_model_dir_iris = os.path.join(tempfile.mkdtemp(), 'serving_model/iris_simple')
_serving_model_dir_mnist = os.path.join(tempfile.mkdtemp(), 'serving_model/mnist_simple')
_serving_model_dir_probablity = os.path.join(tempfile.mkdtemp(), 'serving_model/probablity')

absl.logging.set_verbosity(absl.logging.INFO)

### Download example data
Download the example dataset for use in our TFX pipeline.
* mnist


This example does not use the Estimator API for model training/evaluation.

The trainer must be configured with TFX [Generic Trainer] (https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md).

The dataset included in this example consists of a selection of 1000 records from the MNIST dataset, converted to tfrecord format. Each record is a tf.Example with 2 columns of data: 'image_floats' representing the 28x28 image as 784 float values in the range [-0.5, 0.5], and 'image_class' representing the label with values [0-9] corresponding to decimal digit in the image.

For a detailed description of tf.Example and TFRecord see https://www.tensorflow.org/tutorials/load_data/tfrecord. Several independently written tools can be used to generate the tfrecords from images. See Preparing MNIST data from one such publicly available set of instructions.

In [69]:
!virtualenv -p python3.5 mnist
!source ./mnist/bin/activate

/bin/bash: virtualenv: command not found
/bin/bash: ./mnist/bin/activate: No such file or directory


In [71]:
!git clone https://github.com/tensorflow/tfx ~/tfx-source && pushd ~/tfx-source
!cp -r ~/tfx-source/tfx/examples/mnist ~/

Cloning into '/root/tfx-source'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 31845 (delta 50), reused 58 (delta 30), pack-reused 31734
Receiving objects: 100% (31845/31845), 181.62 MiB | 34.25 MiB/s, done.
Resolving deltas: 100% (23924/23924), done.
~/tfx-source /tmp/tfx-data-mnistf3ddmh9v


In [86]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from typing import List, Text

import absl
import tensorflow_model_analysis as tfma
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ImportExampleGen
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input


In [87]:
_pipeline_name = 'mnist_native_keras'

_mnist_root = os.path.join(os.environ['HOME'], 'mnist')
_data_root = os.path.join(_mnist_root, 'data')

_module_file = os.path.join(_mnist_root, 'mnist_utils_native_keras.py')
_module_file_lite = os.path.join(
    _mnist_root, 'mnist_utils_native_keras_lite.py')


_serving_model_dir = os.path.join(_mnist_root, 'serving_model', _pipeline_name)
_serving_model_dir_lite = os.path.join(
    _mnist_root, 'serving_model_lite', _pipeline_name)

_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)

_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')


_beam_pipeline_args = [
    '--direct_running_mode=multi_processing',
    # 0 means auto-detect based on on the number of CPUs available
    # during execution time.
    '--direct_num_workers=0',
]

In [89]:
def _create_pipeline(pipeline_name: Text, pipeline_root: Text, data_root: Text,
                     module_file: Text, module_file_lite: Text,
                     serving_model_dir: Text, serving_model_dir_lite: Text,
                     metadata_path: Text,
                     beam_pipeline_args: List[Text]) -> pipeline.Pipeline:
  """Implements the handwritten digit classification example using TFX."""
  examples = external_input(data_root)

  # Brings data into the pipeline.
  example_gen = ImportExampleGen(input=examples)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  # Generates schema based on statistics files.
  schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  # Performs anomaly detection based on statistics and data schema.
  example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])

  # Performs transformations and feature engineering in training and serving.
  transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)

  def _create_trainer(module_file, instance_name):
    return Trainer(
        module_file=module_file,
        custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(num_steps=5000),
        eval_args=trainer_pb2.EvalArgs(num_steps=100),
        instance_name=instance_name)

  # Uses user-provided Python function that trains a Keras model.
  trainer = _create_trainer(module_file, 'mnist')

  # Trains the same model as the one above, but converts it into a TFLite one.
  trainer_lite = _create_trainer(module_file_lite, 'mnist_lite')

  # TODO(b/150949276): Add resolver back once it supports two trainers.

  # Uses TFMA to compute an evaluation statistics over features of a model and
  # performs quality validation of a candidate model.
  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='image_class')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.config.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.8})))
          ])
      ])

  eval_config_lite = tfma.EvalConfig()
  eval_config_lite.CopyFrom(eval_config)
  # Informs the evaluator that the model is a TFLite model.
  eval_config_lite.model_specs[0].model_type = 'tf_lite'

  # Uses TFMA to compute the evaluation statistics over features of a model.
  evaluator = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      eval_config=eval_config,
      instance_name='mnist')

  # Uses TFMA to compute the evaluation statistics over features of a TFLite
  # model.
  evaluator_lite = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer_lite.outputs['model'],
      eval_config=eval_config_lite,
      instance_name='mnist_lite')

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir)),
      instance_name='mnist')

  # Checks whether the TFLite model passed the validation steps and pushes the
  # model to a file destination if check passed.
  pusher_lite = Pusher(
      model=trainer_lite.outputs['model'],
      model_blessing=evaluator_lite.outputs['blessing'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir_lite)),
      instance_name='mnist_lite')

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen,
          statistics_gen,
          schema_gen,
          example_validator,
          transform,
          trainer,
          trainer_lite,
          evaluator,
          evaluator_lite,
          pusher,
          pusher_lite,
      ],
      enable_cache=True,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(
          metadata_path),
      beam_pipeline_args=beam_pipeline_args)


In [90]:
absl.logging.set_verbosity(absl.logging.INFO)
pipeline_1=_create_pipeline(pipeline_name=_pipeline_name, pipeline_root=_pipeline_root,
          data_root=_data_root, module_file=_module_file,
          module_file_lite=_module_file_lite, serving_model_dir=_serving_model_dir,
          serving_model_dir_lite=_serving_model_dir_lite,
          metadata_path=_metadata_path, beam_pipeline_args=_beam_pipeline_args)


INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.


In [91]:
BeamDagRunner().run(pipeline_1)

INFO:absl:Component ImportExampleGen depends on [].
INFO:absl:Component ImportExampleGen is scheduled.
INFO:absl:Component StatisticsGen depends on ['Run[ImportExampleGen]'].
INFO:absl:Component StatisticsGen is scheduled.
INFO:absl:Component SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Component SchemaGen is scheduled.
INFO:absl:Component ExampleValidator depends on ['Run[StatisticsGen]', 'Run[SchemaGen]'].
INFO:absl:Component ExampleValidator is scheduled.
INFO:absl:Component Transform depends on ['Run[SchemaGen]', 'Run[ImportExampleGen]'].
INFO:absl:Component Transform is scheduled.
INFO:absl:Component Trainer.mnist depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Component Trainer.mnist is scheduled.
INFO:absl:Component Trainer.mnist_lite depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Component Trainer.mnist_lite is scheduled.
INFO:absl:Component Evaluator.mnist depends on ['Run[Trainer.mnist]', 'Run[ImportExampleGen]'].
INFO:absl:Component Evaluator

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /root/tfx/pipelines/mnist_native_keras/Transform/transform_graph/10/.temp_path/tftransform_tmp/520e963fc40246aa86aecb640f0c9ef7/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /root/tfx/pipelines/mnist_native_keras/Transform/transform_graph/10/.temp_path/tftransform_tmp/df774ff79bc142cdb317b4dd84c10dde/saved_model.pb


INFO:absl:Feature image_class has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature image_floats has a shape dim {
  size: 784
}
. Setting to DenseTensor.


INFO:absl:Feature image_class has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature image_floats has a shape dim {
  size: 784
}
. Setting to DenseTensor.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Transform is finished.
INFO:absl:Component Trainer.mnist_lite is running.
INFO:absl:Running driver for Trainer.mnist_lite
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer.mnist_lite
INFO:absl:Attempting to infer TFX Python dependency for beam
INFO:absl:Copying all content from install dir /usr/local/lib/python3.6/dist-packages/tfx to temp dir /tmp/tmpmfmhet8u/build/tfx
INFO:absl:Generating a temp setup file at /tmp/tmpmfmhet8u/build/tfx/setup.py
INFO:absl:Creating temporary sdist package, logs available at /tmp/tmpmfmhet8u/build/tfx/setup.log
INFO:absl:Added --extra_package=/tmp/tmpmfmhet8u/build/tfx/dist/tfx_ephemeral-0.25.0.tar.gz to beam args
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:Training model.
INFO:absl:Fea

...

INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False
INFO:absl:Training complete. Model written to /root/tfx/pipelines/mnist_native_keras/Trainer.mnist_lite/model/11/serving_model_dir. ModelRun written to /root/tfx/pipelines/mnist_native_keras/Trainer.mnist_lite/model_run/11
INFO:absl:Running publisher for Trainer.mnist_lite
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Trainer.mnist_lite is finished.
INFO:absl:Component Evaluator.mnist_lite is running.
INFO:absl:Running driver for Evaluator.mnist_lite
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator.mnist_lite
INFO:absl:Attempting to infer TFX Python dependency for beam
INFO:absl:Copying all content from install dir /usr/local/lib/python3.6/dist-packages/tfx to temp dir /tmp/tmpazil8n1s/build/tfx
INFO:absl:Generating a temp setup file at /tmp/tmpazil8n1s/build/tfx/se

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:absl:Model: "sequential_1"
INFO:absl:_________________________________________________________________
INFO:absl:Layer (type)                 Output Shape              Param #   
INFO:absl:=================================================================
INFO:absl:dense_3 (Dense)              (None, 64)                50240     
INFO:absl:_________________________________________________________________
INFO:absl:dropout_2 (Dropout)          (None, 64)                0         
INFO:absl:_________________________________________________________________
INFO:absl:dense_4 (Dense)              (None, 64)                4160      
INFO:absl:_________________________________________________________________
INFO:absl:dropout_3 (Dropout)          (None, 64)                0         
INFO:absl:_________________________________________________________________
INFO:absl:dense_5 (Dense)              (None, 10)                650       
INFO:absl:==============================================

5000/5000 [==============================] - 24s 5ms/step - loss: 0.0989 - sparse_categorical_accuracy: 0.9669 - val_loss: 1.6363 - val_sparse_categorical_accuracy: 0.8188
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets written to: /root/tfx/pipelines/mnist_native_keras/Trainer.mnist/model/15/serving_model_dir/assets


INFO:absl:Training complete. Model written to /root/tfx/pipelines/mnist_native_keras/Trainer.mnist/model/15/serving_model_dir. ModelRun written to /root/tfx/pipelines/mnist_native_keras/Trainer.mnist/model_run/15
INFO:absl:Running publisher for Trainer.mnist
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Trainer.mnist is finished.
INFO:absl:Component Evaluator.mnist is running.
INFO:absl:Running driver for Evaluator.mnist
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator.mnist
INFO:absl:Attempting to infer TFX Python dependency for beam
INFO:absl:Copying all content from install dir /usr/local/lib/python3.6/dist-packages/tfx to temp dir /tmp/tmp97d8cawi/build/tfx
INFO:absl:Generating a temp setup file at /tmp/tmp97d8cawi/build/tfx/setup.py
INFO:absl:Creating temporary sdist package, logs available at /tmp/tmp97d8cawi/build/tfx/setup.log
INFO:absl:Added --extra_package=/tmp/tmp97d8cawi/build/tfx/dist/tfx_ephemeral

In [47]:
! git clone https://github.com/tensorflow/tfx.git /tmp/tfx/

Cloning into '/tmp/tfx'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 31845 (delta 50), reused 58 (delta 30), pack-reused 31736
Receiving objects: 100% (31845/31845), 181.63 MiB | 33.32 MiB/s, done.
Resolving deltas: 100% (23925/23925), done.


In [52]:
!ls /tmp/tfx/tfx

benchmarks  dependencies.py  experimental   proto    tools  version.py
BUILD	    dsl		     extensions     scripts  types  workspace.bzl
components  examples	     orchestration  tfx.bzl  utils


In [59]:
! cp /tmp/tfx/tfx/examples/mnist/data/mnist.tfrecord /tmp/tfx-data-mnistf3ddmh9v/mnist.tfrecord

In [61]:
%cd /tmp/tfx-data-mnistf3ddmh9v/

/tmp/tfx-data-mnistf3ddmh9v


In [65]:
!ls

mnist.tfrecord


In [ ]:
!head {_data_filepath_mnist}

In [ ]:
!ls

mnist.tfrecord


### InteractiveContext


In [ ]:
context = InteractiveContext()

## Run TFX components interactively


### Using ImportExampleGen to import the dataset

In [ ]:
from tfx.components import ImportExampleGen
example_gen = ImportExampleGen(input=external_input(_data_root_mnist))
context.run(example_gen)

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

In [ ]:

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')


tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

Now that `ExampleGen` has finished ingesting the data, the next step is data analysis.

### StatisticsGen
ref:https://www.tensorflow.org/tfx/data_validation/get_started

In [ ]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

After `StatisticsGen` finishes running, we can visualize the outputted statistics. Try playing with the different plots!

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen
ref: https://www.tensorflow.org/tfx/data_validation/get_started

In [ ]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],infer_feature_shape=False)
context.run(schema_gen)

In [ ]:
context.show(schema_gen.outputs['schema'])

the SchemaGen documentation:
https://www.tensorflow.org/tfx/guide/schemagen

### ExampleValidator
https://www.tensorflow.org/tfx/data_validation/get_started

In [ ]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

After `ExampleValidator` finishes running, we can visualize the anomalies as a table.

In [ ]:
context.show(example_validator.outputs['anomalies'])

In the anomalies table, we can see that there are no anomalies. This is what we'd expect, since this the first dataset that we've analyzed and the schema is tailored to it. You should review this schema -- anything unexpected means an anomaly in the data. Once reviewed, the schema can be used to guard future data, and anomalies produced here can be used to debug model performance, understand how your data evolves over time, and identify data errors.

### Transform
* https://www.tensorflow.org/tfx/transform/get_started

* https://www.tensorflow.org/tfx/tutorials/transform/simple


In [ ]:
_mnist_constants_module_file = 'mnist_constants.py'

In [ ]:
%%writefile {_mnist_constants_module_file}

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]

CATEGORICAL_FEATURE_KEYS = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

DENSE_FLOAT_FEATURE_KEYS = ['trip_miles', 'fare', 'trip_seconds']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = [
    'payment_type',
    'company',
]

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

def transformed_name(key):
  return key + '_xf'

Next, we write a `preprocessing_fn` that takes in raw data as input, and returns transformed features that our model can train on:

In [ ]:
_mnist_transform_module_file = 'mnist_transform.py'

In [ ]:
%%writefile {_mnist_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import mnist_constants

_DENSE_FLOAT_FEATURE_KEYS = mnist_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = mnist_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = mnist_constants.VOCAB_SIZE
_OOV_SIZE = mnist_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = mnist_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = mnist_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = mnist_constants.CATEGORICAL_FEATURE_KEYS
_FARE_KEY = mnist_constants.FARE_KEY
_LABEL_KEY = mnist_constants.LABEL_KEY
_transformed_name = mnist_constants.transformed_name


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        _fill_in_missing(inputs[key]),
        top_k=_VOCAB_SIZE,
        num_oov_buckets=_OOV_SIZE)

  for key in _BUCKET_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.bucketize(
        _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT)

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

  # Was this passenger a big tipper?
  mnist_fare = _fill_in_missing(inputs[_FARE_KEY])
  tips = _fill_in_missing(inputs[_LABEL_KEY])
  outputs[_transformed_name(_LABEL_KEY)] = tf.where(
      tf.math.is_nan(mnist_fare),
      tf.cast(tf.zeros_like(mnist_fare), tf.int64),
      # Test if the tip was > 20% of the fare.
      tf.cast(
          tf.greater(tips, tf.multiply(mnist_fare, tf.constant(0.2))), tf.int64))

  return outputs


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Now, we pass in this feature engineering code to the `Transform` component and run it to transform your data.

In [ ]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_mnist_transform_module_file))
context.run(transform)

Let's examine the output artifacts of `Transform`. This component produces two types of outputs:

* `transform_graph` is the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
* `transformed_examples` represents the preprocessed training and evaluation data.

In [ ]:
transform.outputs

Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [ ]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

The `transformed_metadata` subdirectory contains the schema of the preprocessed data. The `transform_fn` subdirectory contains the actual preprocessing graph. The `metadata` subdirectory contains the schema of the original data.

We can also take a look at the first three transformed examples:

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

After the `Transform` component has transformed your data into features, and the next step is to train a model.

### Trainer
The `Trainer` component will train a model that you define in TensorFlow. Default Trainer support Estimator API, to use Keras API, you need to specify [Generic Trainer](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md) by setup `custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)` in Trainer's contructor.

`Trainer` takes as input the schema from `SchemaGen`, the transformed data and graph from `Transform`, training parameters, as well as a module that contains user-defined model code.

Let's see an example of user-defined model code below (for an introduction to the TensorFlow Keras APIs, [see the tutorial](https://www.tensorflow.org/guide/keras)):

In [ ]:
_mnist_trainer_module_file = 'mnist_trainer.py'

In [ ]:
%%writefile {_mnist_trainer_module_file}

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options

import mnist_constants

_DENSE_FLOAT_FEATURE_KEYS = mnist_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = mnist_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = mnist_constants.VOCAB_SIZE
_OOV_SIZE = mnist_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = mnist_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = mnist_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = mnist_constants.CATEGORICAL_FEATURE_KEYS
_MAX_CATEGORICAL_FEATURE_VALUES = mnist_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = mnist_constants.LABEL_KEY
_transformed_name = mnist_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying mnist data.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_VOCAB_SIZE + _OOV_SIZE, default_value=0)
      for key in _transformed_names(_VOCAB_FEATURE_KEYS)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_FEATURE_BUCKET_COUNT, default_value=0)
      for key in _transformed_names(_BUCKET_FEATURE_KEYS)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_CATEGORICAL_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]
  indicator_column = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
  ]

  model = _wide_and_deep_classifier(
      # TODO(b/139668410) replace with premade wide_and_deep keras model
      wide_columns=indicator_column,
      deep_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25])
  return model


def _wide_and_deep_classifier(wide_columns, deep_columns, dnn_hidden_units):
  """Build a simple keras wide and deep model.

  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Wide and Deep Keras model
  """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.
  # TODO(b/139081439): Automate generation of input layers from FeatureColumn.
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_CATEGORICAL_FEATURE_KEYS)
  })

  # TODO(b/161952382): Replace with Keras preprocessing layers.
  deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
  for numnodes in dnn_hidden_units:
    deep = tf.keras.layers.Dense(numnodes)(deep)
  wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

  output = tf.keras.layers.Dense(
      1, activation='sigmoid')(
          tf.keras.layers.concatenate([deep, wide]))

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])
  model.summary(print_fn=absl.logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  # Number of nodes in the first layer of the DNN
  first_dnn_layer_size = 100
  num_dnn_layers = 4
  dnn_decay_factor = 0.7

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, 40)

  model = _build_keras_model(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Now, we pass in this model code to the `Trainer` component and run it to train the model.

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(_mnist_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)

#### Analyze Training with TensorBoard
Take a peek at the trainer artifact. It points to a directory containing the model subdirectories.

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
pp.pprint(os.listdir(model_dir))

Optionally, we can connect TensorBoard to the Trainer to analyze our model's training curves.

In [ ]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

### Evaluator


In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key='tips')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(feature_keys=['trip_start_hour'])
    ])

Next, we give this configuration to `Evaluator` and run it.

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run).
    eval_config=eval_config)
context.run(evaluator)

Now let's examine the output artifacts of `Evaluator`. 

In [ ]:
evaluator.outputs

Using the `evaluation` output we can show the default visualization of global metrics on the entire evaluation set.

In [ ]:
context.show(evaluator.outputs['evaluation'])

To see the visualization for sliced evaluation metrics, we can directly call the TensorFlow Model Analysis library.

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='trip_start_hour')

This visualization shows the same metrics, but computed at every feature value of `trip_start_hour` instead of on the entire evaluation set.

TensorFlow Model Analysis supports many other visualizations, such as Fairness Indicators and plotting a time series of model performance. To learn more, see [the tutorial](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

Since we added thresholds to our config, validation output is also available. The precence of a `blessing` artifact indicates that our model passed validation. Since this is the first validation being performed the candidate is automatically blessed.

In [ ]:
blessing_uri = evaluator.outputs.blessing.get()[0].uri
!ls -l {blessing_uri}

Now can also verify the success by loading the validation result record:

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

### Pusher
The `Pusher` component is usually at the end of a TFX pipeline. It checks whether a model has passed validation, and if so, exports the model to `_serving_model_dir`.

# New Section

In [ ]:
pusher = Pusher(  model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir_mnist)))
context.run(pusher)

Let's examine the output artifacts of `Pusher`. 

In [ ]:
pusher.outputs

In [ ]:
push_uri = pusher.outputs.model_push.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

We're finished our tour of built-in TFX components!